In [1]:
import numpy as np

In [11]:
np.random.seed(42)
# Exemple de forme de Z et b
m, n, p = 3, 4, 2  # Nombre d'éléments dans le batch et nombre de neurones dans la couche Dense
Z = np.random.rand(m, n)  # Sortie de la couche Dense
b = np.random.rand(n)  # Vecteur de biais
#print(b)
# print(np.diagflat(b))
#print(Z.shape, Z)
#print(b.shape, b)
# Addition avec biais
#output = Z + b
c = np.mean(Z, axis=0)
print(np.dot(b, 0)+None)

TypeError: unsupported operand type(s) for +: 'float' and 'NoneType'

In [ ]:
print([1, 2, 3]+[-1,-1,-1])